## **DATASET CREATION**

In [62]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from langdetect import detect
import re
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from langdetect import detect
import spacy
from tqdm import tqdm
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
import os


In [63]:
# Define a dictionary with file paths and metadata
politicians_data = {
    'FdI_Meloni': {'file': 'politicians_data/GiorgiaMeloni_tweets_combined.csv', 'politician': 'Meloni', 'party': 'FdI'},
    'FdI_LaRussa': {'file': 'politicians_data/IgnazioLaRussa_tweets_combined.csv', 'politician': 'LaRussa', 'party': 'FdI'},
    'FI_Berlusconi': {'file': 'politicians_data/SilvioBerlusconi_tweets_combined.csv', 'politician': 'Berlusconi', 'party': 'FI'},
    'FI_Tajani': {'file': 'politicians_data/AntonioTajani_tweets_combined.csv', 'politician': 'Tajani', 'party': 'FI'},
    'Lega_Salvini': {'file': 'politicians_data/MatteoSalvini_tweets_combined.csv', 'politician': 'Salvini', 'party': 'Lega'},
    'M5S_DiMaio': {'file': 'politicians_data/luigidimaio_tweets_combined.csv', 'politician': 'Di Maio', 'party': 'M5S'},
    'M5S_Conte': {'file': 'politicians_data/GiuseppeConte_tweets_combined.csv', 'politician': 'Conte', 'party': 'M5S'},
    'Az_Calenda': {'file': 'politicians_data/CarloCalenda_tweets_combined.csv', 'politician': 'Calenda', 'party': 'Azione'},
    'IV_Renzi': {'file': 'politicians_data/MatteoRenzi_tweets_combined.csv', 'politician': 'Renzi', 'party': 'IV'},
    'PEeur_Bonino': {'file': 'politicians_data/emmabonino_tweets_combined.csv', 'politician': 'Bonino', 'party': 'PEeur'},
    'PD_Shlein': {'file': 'politicians_data/EllySchlein_tweets_combined.csv', 'politician': 'Schlein', 'party': 'PD'},
    'PD_Letta': {'file': 'politicians_data/EnricoLetta_tweets_combined.csv', 'politician': 'Letta', 'party': 'PD'},
    'EV_Fratoianni': {'file': 'politicians_data/NicolaFratoianni_tweets_combined.csv', 'politician': 'Fratoianni', 'party': 'EV'},
    'NcI_Lupi': {'file': 'politicians_data/MaurizioLupi_tweets_combined.csv', 'politician': 'Lupi', 'party': 'NcI'}
}

# Read and process each file
politicians_list = []
for key, data in politicians_data.items():
    df = pd.read_csv(data['file'])
    df['politician'] = data['politician']
    df['party'] = data['party']
    politicians_list.append(df)

# Concatenate all DataFrames into one
politicians = pd.concat(politicians_list, ignore_index=True)

# Load the users dataset
users = pd.read_csv('train_data/user_tweets.csv')


In [64]:
#how can i see the different values in the ideology_multiclass column?
users['ideology_multiclass']= users['ideology_multiclass'].astype('category')
dict(enumerate(users['ideology_multiclass'].cat.categories))

{0: 'left', 1: 'moderate_left', 2: 'moderate_right', 3: 'right'}

In [65]:
#let me delete the urls column and the retweet column
politicians = politicians.drop(columns=[politicians.columns[2], politicians.columns[5]])

In [66]:
politicians.head()

,Date,ID,Content,Likes,politician,party
0,2022-12-30 15:30:11+00:00,1608847928625434625,Ecco l’ultimo appuntamento con #gliappuntidiGi...,3385,Meloni,FdI
1,2022-12-29 20:55:30+00:00,1608567407500754944,Congratulations to @netanyahu on the formation...,6021,Meloni,FdI
2,2022-12-29 19:48:50+00:00,1608550629005488130,Grazie al suo estro e alla sua classe è riusci...,32003,Meloni,FdI
3,2022-12-29 17:31:05+00:00,1608515965041651712,Nel corso della conferenza stampa di fine anno...,4146,Meloni,FdI
4,2022-12-29 10:38:45+00:00,1608412198182830081,Conferenza stampa di fine anno. Seguitemi in d...,3523,Meloni,FdI


In [67]:
#i want to use this map now to also add the ideology_multiclass column to the politicians dataframe
politicians_parties = {
    'Meloni': 3,  # 'right'
    'Salvini': 3,  # 'right'
    'Berlusconi': 2,  # 'moderate_right'
    'Letta': 1,  # 'moderate_left'
    'Conte': 1,  # 'moderate_left'
    'Calenda': 2,  # 'moderate_right'
    'Renzi': 2,  # 'moderate_right'
    'Bonino': 2,  # 'moderate_right'
    'Di Maio': 1,  # 'moderate_left'
    'Fratoianni': 0,  # 'left'
    'Bonelli': 0,  # 'left'
    'Lupi': 3,  # 'right'
    'La Russa': 3,  # 'right'
    'Tajani': 2,  # 'moderate_right'
    'Schlein': 0   # 'left'
}
politicians['ideology_num'] = politicians['politician'].map(politicians_parties)
politicians['ideology_num'].value_counts()

ideology_num
2.0    13383
3.0     8217
1.0     3555
0.0     2260
Name: count, dtype: int64

In [68]:
users['ideology_multiclass'].cat.codes
#let's check the ideology_multiclass column
users_map={ 'left':0, 'moderate_left':1, 'moderate_right':2,'right':3}
users['ideology_num']=users['ideology_multiclass'].map(users_map)
users['ideology_num'].value_counts()

ideology_num
1    44640
3    35760
0    12960
2    10480
Name: count, dtype: int64

In [69]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load stopwords
stop_words_italian = set(stopwords.words('italian'))
stop_words_english = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Load spaCy models for Italian and English
# You need to install these models first using:
nlp_it = spacy.load('it_core_news_sm')
nlp_en = spacy.load('en_core_web_sm')

def preprocess_text(text):
    try:
        # Detect language
        lang = detect(text)
    except:
        lang = 'unknown'
    
    # Remove URLs, mentions, hashtags, and punctuation
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    if lang == 'it':  # Italian
        # Process with spaCy for Italian
        doc = nlp_it(text.lower())
        # Get lemmas and filter stopwords and short words
        true_tokens = [token.lemma_ for token in doc 
                      if token.text not in stop_words_italian and len(token.text) > 2]
    
    elif lang == 'en':  # English
        # Process with spaCy for English
        doc = nlp_en(text.lower())
        # Get lemmas and filter stopwords and short words
        true_tokens = [token.lemma_ for token in doc 
                      if token.text not in stop_words_english and len(token.text) > 2]
    
    else: 
        # Fallback to NLTK for spacy errors or mislablings
        tokens = word_tokenize(text.lower())
        true_tokens = [lemmatizer.lemmatize(word) for word in tokens if len(word) > 2]
    
    return true_tokens

# Apply preprocessing to the tweets
politicians_tweets = politicians['Content']

# Method 1: Using tqdm with apply (simpler)
tqdm.pandas(desc="Processing tweets")
politicians['processed_tweet'] = politicians_tweets.progress_apply(lambda x: preprocess_text(x))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/filippofocaccia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/filippofocaccia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/filippofocaccia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Processing tweets: 100%|██████████| 27865/27865 [07:01<00:00, 66.06it/s]


### Let's perform topic modeling on all the politicians' tweets using LDA, initially we perform it on the whole dataset

In [70]:
dictionary = corpora.Dictionary(politicians['processed_tweet'])
corpus = [dictionary.doc2bow(text) for text in politicians['processed_tweet']]


In [71]:
num_topics = 5  # Numero di topic da identificare

In [72]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)

In [73]:
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.008*"legge" + 0.007*"vaccino" + 0.005*"presidente" + 0.005*"parlamentare" + 0.004*"parlamento" + 0.004*"giustizia" + 0.004*"drago" + 0.004*"firmare" + 0.004*"ddlzan" + 0.004*"parere"
Topic 1: 0.014*"fare" + 0.011*"potere" + 0.011*"governo" + 0.011*"dovere" + 0.010*"essere" + 0.009*"dire" + 0.007*"solo" + 0.007*"paese" + 0.007*"volere" + 0.007*"cosa"
Topic 2: 0.031*"ora" + 0.016*"ospite" + 0.016*"oggi" + 0.010*"diretta" + 0.010*"aspettare" + 0.010*"qui" + 0.010*"sinistraitaliano" + 0.009*"Roma" + 0.008*"poco" + 0.007*"governodrago"
Topic 3: 0.014*"grazie" + 0.011*"anno" + 0.010*"oggi" + 0.009*"bello" + 0.008*"grande" + 0.007*"tanto" + 0.007*"donna" + 0.006*"essere" + 0.006*"buono" + 0.006*"tutto"
Topic 4: 0.012*"lavoratore" + 0.011*"lavoro" + 0.011*"salvino" + 0.009*"impresa" + 0.006*"famiglia" + 0.006*"Putin" + 0.006*"giovane" + 0.005*"   " + 0.005*"tassa" + 0.005*"gas"


In [74]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_total'+str(num_topics))

if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_total'+ str(num_topics) +'.html')

LDAvis_prepared

/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/filippofocaccia/anaconda3/lib/python3.11/site-packages/pandas/cor

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.217453  0.032310       1        1  44.705339
3     -0.165085 -0.001654       2        1  21.318630
4      0.228194  0.228481       3        1  11.781687
2      0.167054 -0.270794       4        1  11.216554
0     -0.012710  0.011656       5        1  10.977790, topic_info=            Term         Freq        Total Category  logprob  loglift
371          ora  2401.000000  2401.000000  Default  30.0000  30.0000
28        grazie  1324.000000  1324.000000  Default  29.0000  29.0000
2198      ospite   765.000000   765.000000  Default  28.0000  28.0000
33          oggi  2508.000000  2508.000000  Default  27.0000  27.0000
1038  lavoratore   599.000000   599.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2675       drago   188.277354  1135.108809   Topic5  -5.5195   0.4127
427     ministro   129.194766   572.638603   Topic5  -5.8961   0.7204
102         ogni   110.784849   766.858239   Topic5  -6.0498   0.2746
199      europeo   108.197540   652.964637   Topic5  -6.0735   0.4117
1547    scrivere   101.124762   343.616783   Topic5  -6.1411   0.9861

[297 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2314      4  0.994963      \n\n
6793      3  0.995360          
3104      4  0.993541      2021
320       4  0.994919   Bologna
2242      4  0.995140  Facebook
...     ...       ...       ...
61        2  0.137826    volere
61        3  0.000632    volere
61        5  0.007587    volere
1480      1  0.946364      voto
1480      5  0.052667      voto

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 3, 1])

### Then, we divide the politicians in separate datasets with respect to their belief

In [75]:
right_wing=politicians[politicians['ideology_num'] == 3]
left_wing=politicians[politicians['ideology_num'] == 0]
moderate_right=politicians[politicians['ideology_num'] == 2]
moderate_left=politicians[politicians['ideology_num'] == 1]


### Let's perform topic modeling now on right wing politicians

In [76]:
dictionary_right = corpora.Dictionary(right_wing['processed_tweet'])
corpus_right = [dictionary_right.doc2bow(text) for text in right_wing['processed_tweet']]
# 4. Applicazione di LDA
lda_model_right= LdaModel(corpus=corpus_right, id2word=dictionary_right, num_topics=num_topics, passes=10, random_state=42)
# 6. Stampa dei topic
for idx, topic in lda_model_right.print_topics(-1):
    print(f"Topic {idx}: {topic}")



Topic 0: 0.012*"italiano" + 0.010*"anno" + 0.007*"grande" + 0.006*"oggi" + 0.006*"essere" + 0.006*"grazie" + 0.006*"vita" + 0.005*"buono" + 0.004*"mondo" + 0.004*"libertà"
Topic 1: 0.016*"diretta" + 0.014*"Roma" + 0.011*"amico" + 0.010*"buono" + 0.009*"qui" + 0.008*"grazie" + 0.007*"oggi" + 0.007*"sindaco" + 0.007*"aspettare" + 0.006*"referendumgiustizia"
Topic 2: 0.010*"scuola" + 0.008*"lega" + 0.006*"dovere" + 0.006*"essere" + 0.006*"persona" + 0.005*"solo" + 0.005*"sempre" + 0.005*"anno" + 0.004*"giorno" + 0.004*"solidarietà"
Topic 3: 0.017*"governo" + 0.010*"potere" + 0.010*"dovere" + 0.009*"impresa" + 0.009*"lavoro" + 0.008*"lega" + 0.007*"drago" + 0.007*"famiglia" + 0.007*"centrodestra" + 0.006*"bastare"
Topic 4: 0.011*"potere" + 0.010*"dovere" + 0.010*"fare" + 0.009*"essere" + 0.007*"italiano" + 0.007*"ministro" + 0.006*"volere" + 0.006*"salvino" + 0.006*"governo" + 0.006*"sinistra"


In [77]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_right'+str(num_topics))

if 1 == 1:
    LDAvis_prepared_right = gensimvis.prepare(lda_model_right, corpus_right, dictionary_right)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared_right, './results/ldavis_prepared_right'+ str(num_topics) +'.html')

LDAvis_prepared_right

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.217453  0.032310       1        1  44.705339
3     -0.165085 -0.001654       2        1  21.318630
4      0.228194  0.228481       3        1  11.781687
2      0.167054 -0.270794       4        1  11.216554
0     -0.012710  0.011656       5        1  10.977790, topic_info=            Term         Freq        Total Category  logprob  loglift
371          ora  2401.000000  2401.000000  Default  30.0000  30.0000
28        grazie  1324.000000  1324.000000  Default  29.0000  29.0000
2198      ospite   765.000000   765.000000  Default  28.0000  28.0000
33          oggi  2508.000000  2508.000000  Default  27.0000  27.0000
1038  lavoratore   599.000000   599.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2675       drago   188.277354  1135.108809   Topic5  -5.5195   0.4127
427     ministro   129.194766   572.638603   Topic5  -5.8961   0.7204
102         ogni   110.784849   766.858239   Topic5  -6.0498   0.2746
199      europeo   108.197540   652.964637   Topic5  -6.0735   0.4117
1547    scrivere   101.124762   343.616783   Topic5  -6.1411   0.9861

[297 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2314      4  0.994963      \n\n
6793      3  0.995360          
3104      4  0.993541      2021
320       4  0.994919   Bologna
2242      4  0.995140  Facebook
...     ...       ...       ...
61        2  0.137826    volere
61        3  0.000632    volere
61        5  0.007587    volere
1480      1  0.946364      voto
1480      5  0.052667      voto

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 5, 3, 1])

### Let's now proceed with left wing politicians

In [78]:
dictionary_left = corpora.Dictionary(left_wing['processed_tweet'])
corpus_left = [dictionary_left.doc2bow(text) for text in left_wing['processed_tweet']]
lda_model_left = LdaModel(corpus=corpus_left, id2word=dictionary_left, num_topics=num_topics, passes=10, random_state=42)
# 6. Stampa dei topic
for idx, topic in lda_model_left.print_topics(-1):
    print(f"Topic {idx}: {topic}")


Topic 0: 0.010*"dovere" + 0.008*"essere" + 0.007*"fare" + 0.007*"lavoratore" + 0.007*"lavoro" + 0.005*"paese" + 0.005*"cèchidiceno" + 0.005*"diritto" + 0.004*"nuovo" + 0.004*"anno"
Topic 1: 0.014*"governo" + 0.009*"fare" + 0.008*"vaccino" + 0.008*"paese" + 0.008*"italiano" + 0.007*"covid19" + 0.006*"oggi" + 0.006*"pandemia" + 0.005*"potere" + 0.005*"bene"
Topic 2: 0.040*"ora" + 0.037*"ospite" + 0.012*"governodrago" + 0.012*"vaccino" + 0.008*"covid19" + 0.008*"sinistraitaliano" + 0.006*"mercoledi" + 0.006*"sinistra" + 0.006*"governo" + 0.006*"crisidigoverno"
Topic 3: 0.010*"lavoro" + 0.008*"giorno" + 0.007*"oggi" + 0.007*"governo" + 0.007*"volere" + 0.006*"diritto" + 0.006*"potere" + 0.006*"rispondere" + 0.006*"sinistra" + 0.006*"lavoratore"
Topic 4: 0.016*"governo" + 0.009*"ora" + 0.008*"essere" + 0.007*"dire" + 0.006*"potere" + 0.006*"dopo" + 0.006*"scuola" + 0.006*"covid19" + 0.005*"italiano" + 0.005*"diritto"


In [87]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_left'+str(num_topics))

if 1 == 1:
    LDAvis_prepared_left = gensimvis.prepare(lda_model_left, corpus_left, dictionary_left)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared_left, './results/ldavis_prepared_left'+ str(num_topics) +'.html')

LDAvis_prepared_left

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.050360  0.039732       1        1  28.642706
4      0.067546 -0.027646       2        1  21.056937
1      0.024666 -0.164879       3        1  19.193375
0      0.077323  0.126276       4        1  16.917856
2     -0.219896  0.026516       5        1  14.189126, topic_info=            Term        Freq       Total Category  logprob  loglift
135       ospite  193.000000  193.000000  Default  30.0000  30.0000
217          ora  375.000000  375.000000  Default  29.0000  29.0000
2209     vaccino  117.000000  117.000000  Default  28.0000  28.0000
6         lavoro  155.000000  155.000000  Default  27.0000  27.0000
994   rispondere   63.000000   63.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
448       scuola   23.036496   97.986404   Topic5  -5.4172   0.5049
247      governo   30.303891  341.712846   Topic5  -5.1430  -0.4700
24        essere   24.392772  211.098598   Topic5  -5.3600  -0.2053
1067      dovere   18.664359  161.165512   Topic5  -5.6277  -0.2031
16         tempo   14.153636   64.830183   Topic5  -5.9043   0.4309

[364 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
8131      1  0.931087  17febbraio
4452      4  0.934691     18marzo
8201      5  0.918940   26gennaio
4085      2  0.941972      3marzo
4559      1  0.958238      8marzo
...     ...       ...         ...
1034      3  0.160812       volta
1034      4  0.137839       volta
934       2  0.068258      votare
934       3  0.238904      votare
934       5  0.682583      votare

[613 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 1, 3])

### Let's now proceed with moderate right

In [80]:
dictionary_moderate_right = corpora.Dictionary(moderate_right['processed_tweet'])
corpus_moderate_right = [dictionary_moderate_right.doc2bow(text) for text in moderate_right['processed_tweet']]
lda_model_mod_right = LdaModel(corpus=corpus_moderate_right, id2word=dictionary_moderate_right, num_topics=num_topics, passes=10, random_state=42)
# 6. Stampa dei topic
for idx, topic in lda_model_mod_right.print_topics(-1):
    print(f"Topic {idx}: {topic}")


Topic 0: 0.006*"anno" + 0.006*"molto" + 0.005*"bravo" + 0.005*"ragione" + 0.005*"cinque" + 0.004*"essere" + 0.004*"sovranista" + 0.004*"municipio" + 0.004*"maio" + 0.004*"dato"
Topic 1: 0.020*"oggi" + 0.012*"grazie" + 0.011*"diretta" + 0.009*"aspettare" + 0.008*"Lenews" + 0.008*"vedere" + 0.008*"bello" + 0.008*"poco" + 0.007*"buono" + 0.006*"settimana"
Topic 2: 0.007*"dovere" + 0.007*"lavoro" + 0.006*"fare" + 0.006*"vaccino" + 0.006*"europeo" + 0.005*"oggi" + 0.005*"piano" + 0.004*"potere" + 0.004*"essere" + 0.004*"anno"
Topic 3: 0.016*"fare" + 0.013*"dire" + 0.011*"cosa" + 0.010*"potere" + 0.009*"essere" + 0.009*"drago" + 0.009*"governo" + 0.008*"volere" + 0.008*"politica" + 0.008*"solo"
Topic 4: 0.013*"Roma" + 0.007*"candidato" + 0.007*"fare" + 0.006*"lavoro" + 0.005*"grande" + 0.005*"persona" + 0.005*"dovere" + 0.005*"essere" + 0.005*"buono" + 0.004*"solo"


In [86]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_mod_right'+str(num_topics))

if 1 == 1:
    LDAvis_prepared_mod_right = gensimvis.prepare(lda_model_mod_right, corpus_moderate_right, dictionary_moderate_right)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared_mod_right, './results/ldavis_prepared_mod_right'+ str(num_topics) +'.html')

LDAvis_prepared_mod_right

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.194601 -0.022557       1        1  35.470752
2      0.087036 -0.017324       2        1  22.593918
4      0.057025 -0.016486       3        1  18.356916
1     -0.212830 -0.160124       4        1  12.783328
0     -0.125832  0.216491       5        1  10.795087, topic_info=         Term         Freq        Total Category  logprob  loglift
275      oggi  1163.000000  1163.000000  Default  30.0000  30.0000
435      dire  1091.000000  1091.000000  Default  29.0000  29.0000
22       cosa   809.000000   809.000000  Default  28.0000  28.0000
2271  diretta   284.000000   284.000000  Default  27.0000  27.0000
47     grazie   526.000000   526.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
440   libertà    50.645749   251.482626   Topic5  -6.0632   0.6236
346     parte    52.849267   360.718291   Topic5  -6.0207   0.3054
655     tutto    49.617234   406.011200   Topic5  -6.0838   0.1240
262    sempre    50.324133   578.342445   Topic5  -6.0696  -0.2156
926      Roma    48.806010   710.601181   Topic5  -6.1002  -0.4522

[339 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3652      2  0.990550         2021
4785      3  0.974416  25settembre
3352      4  0.991772     Facebook
0         1  0.390422       Italia
0         2  0.365307       Italia
...     ...       ...          ...
1131      3  0.342093       votare
1131      5  0.004358       votare
1094      1  0.794691         voto
1094      3  0.204170         voto
7706      3  0.979806   zingaretto

[580 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 2, 1])

### We now perform it on moderate left

In [82]:
dictionary_moderate_left = corpora.Dictionary(moderate_left['processed_tweet'])
corpus_moderate_left = [dictionary_moderate_left.doc2bow(text) for text in moderate_left['processed_tweet']]
lda_model_mod_left = LdaModel(corpus=corpus_moderate_left, id2word=dictionary_moderate_left, num_topics=num_topics, passes=10, random_state=42)
# 6. Stampa dei topic
for idx, topic in lda_model_mod_left.print_topics(-1):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.012*"oggi" + 0.008*"anno" + 0.007*"giorno" + 0.006*"grazie" + 0.006*"tanto" + 0.006*"grande" + 0.006*"sempre" + 0.005*"tutto" + 0.005*"ancora" + 0.004*"paese"
Topic 1: 0.010*"oggi" + 0.009*"lavoro" + 0.007*"dovere" + 0.006*"governo" + 0.006*"fare" + 0.006*"dare" + 0.005*"europeo" + 0.005*"presidente" + 0.005*"paese" + 0.005*"potere"
Topic 2: 0.009*"diretta" + 0.007*"oggi" + 0.006*"grande" + 0.006*"poco" + 0.005*"bello" + 0.005*"domani" + 0.004*"vedere" + 0.004*"ora" + 0.004*"primo" + 0.003*"nuovo"
Topic 3: 0.005*"dovere" + 0.005*"guerra" + 0.004*"fare" + 0.004*"fermare" + 0.004*"famiglia" + 0.004*"voto" + 0.003*"oggi" + 0.003*"ora" + 0.003*"potere" + 0.003*"italiano"
Topic 4: 0.010*"essere" + 0.009*"oggi" + 0.006*"ora" + 0.005*"avanti" + 0.004*"volere" + 0.004*"potere" + 0.004*"tanto" + 0.004*"grazie" + 0.004*"intervista" + 0.004*"ieri"


In [85]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_mod_left'+str(num_topics))

if 1 == 1:
    LDAvis_prepared_mod_left = gensimvis.prepare(lda_model_mod_left, corpus_moderate_left, dictionary_moderate_left)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared_mod_left, './results/ldavis_prepared_mod_left'+ str(num_topics) +'.html')

LDAvis_prepared_mod_left

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.105937 -0.053785       1        1  32.754099
0     -0.063492  0.039650       2        1  20.610371
4     -0.051618  0.002735       3        1  20.006689
3      0.127045 -0.124409       4        1  13.608480
2      0.094002  0.135809       5        1  13.020361, topic_info=           Term        Freq       Total Category  logprob  loglift
41      diretta   65.000000   65.000000  Default  30.0000  30.0000
129        anno  119.000000  119.000000  Default  29.0000  29.0000
371      lavoro  178.000000  178.000000  Default  28.0000  28.0000
777      domani   75.000000   75.000000  Default  27.0000  27.0000
611      guerra   68.000000   68.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
280   scegliere   14.141128   80.416260   Topic5  -6.0779   0.3005
731     sindaco   13.147458   56.954004   Topic5  -6.1507   0.5726
329      futuro   13.187095   86.500076   Topic5  -6.1477   0.1577
251        dire   13.456695  128.100379   Topic5  -6.1275  -0.2147
202  importante   12.601865   76.171239   Topic5  -6.1931   0.2395

[379 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5100      2  0.073369         
5100      3  0.146738         
5100      5  0.733691         
6222      5  0.827233         
2461      1  0.885333  1maggio
...     ...       ...      ...
36        3  0.272007     voto
36        4  0.435211     voto
36        5  0.126937     voto
1372      4  0.844697      war
1633      2  0.937237     zaki

[725 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 4, 3])